In [71]:
from numpy import *
from matplotlib import *
from pylab import *
from scipy import *
from numpy import*
import csv
import xlrd

rc('text',usetex = True)
rc('font', family='serif')

from scipy.optimize import curve_fit
from scipy import integrate

In [72]:
workbook = xlrd.open_workbook('Tc_290.xlsx')
Prop = workbook.sheet_by_name('Prop')

In [73]:
N =len(Prop.col(0))

T_v = []
B_v = []
cp_v = []
dS_v = []
dT_v = []
M_v = []

for i in range(N):
    T_v.append(Prop.cell(i,0).value)
    B_v.append(Prop.cell(i,1).value)
    cp_v.append(Prop.cell(i,2).value)
    dS_v.append(Prop.cell(i,3).value)
    dT_v.append(Prop.cell(i,4).value)
    M_v.append(Prop.cell(i,5).value)

In [74]:
T = np.arange(242.5,309.59,0.1)
B = np.arange(0,2.001,0.01)

In [75]:
cp_m = zeros([len(B),len(T)])
dS_m = zeros([len(B),len(T)])
dT_m = zeros([len(B),len(T)])
M_m  = zeros([len(B),len(T)])

cp_a = np.array(cp_v)
dS_a = np.array(dS_v)
dT_a = np.array(dT_v)
M_a = np.array(M_v)

for k in range(len(B)):
    cp_m[k] = cp_a[len(T)*k:len(T)*(k+1)]
    dS_m[k] = dS_a[len(T)*k:len(T)*(k+1)]
    dT_m[k] = dT_a[len(T)*k:len(T)*(k+1)]
    M_m[k] = M_a[len(T)*k:len(T)*(k+1)]      

In [6]:
def Prop_int(Prop,T,B,T_int,B_int):
    i = 0
    i_int = 0
    while T_int > T[i]:
            i_int = i
            i = i+1
    j = 0 
    j_int = 0
    while B_int > B[j]:
            j_int = j
            j = j+1
                   
    Prop_int  = Prop[j_int+1][i_int] 
    return Prop_int

In [7]:
def H_eff(H,Tc,Th,n,M):
    N_cas = (2*n/np.sqrt(np.pi))*(2*(2*n/(np.sqrt(np.pi)))+1)**-1
    N_t = 1/3 + (1-0.36)*(N_cas-1/3)
    T_e = np.arange(Tc,Th+0.01,0.1)
    M_eff = zeros([len(H),len(T_e)]) 
    H_eff = zeros([len(H),len(T_e)])
    for i in range(len(H)):
        for j in range(len(T_e)):
            M_eff[i][j] = Prop_int(M_m,T,B,T_e[j],H[i])*4*np.pi*10**-7
            #print(T_e[j],H[i],M_eff[i][j])
            H_eff[i][j] = H[i] - N_t*M_eff[i][j]       
    return(H_eff)

In [8]:
tau = np.arange(0,1,0.01)
H = 1.69*np.abs(np.sin(pi*tau+pi/4))
H = [0.63,1.52]
T_h = np.arange(270,300,0.1)
T_dm = T+dT_m

In [9]:
H_e = H_eff(H,270,300,0.1/0.0224,M_m)

In [11]:
def Mean_C(Tc,Th):

    T_p = np.arange(Tc,Th+0.01,0.1)

    C_low = []
    C_high = []
    for i in range(len(T_p)):
        C_low.append(Prop_int(cp_m,T,B,T_p[i],H_e[0][i]))
        C_high.append(Prop_int(cp_m,T,B,T_p[i],H_e[1][i]))

    C_low_mean = integrate.simps(C_low,T_p)/(Th-Tc)
    C_high_mean = integrate.simps(C_high,T_p)/(Th-Tc)
    return(C_low_mean,C_high_mean)

In [61]:
def Mean_dT(Tc,Th):
    T_p = np.arange(Tc,Th+0.01,0.1)
    
    dT_low_mag = []
    dT_high_mag = []
    dT_low_dmag = []
    dT_high_dmag = []

    
    for i in range(len(T_p)):
        j = 0
        j_l = 0
        
        while H_e[0][i] > B[j]:
            j_l = j
            j = j+1
    
        j_h = 0
        
        while H_e[1][i] > B[j]:
            j_h = j
            j = j+1
            
        dT_low_mag.append(Prop_int(dT_m,T,B,T_p[i],H_e[0][i]))
        dT_high_mag.append(Prop_int(dT_m,T,B,T_p[i],H_e[1][i]))
        dT_low_dmag.append(Prop_int(dT_m,T_dm[j_l],B,T_p[i],H_e[0][i]))
        dT_high_dmag.append(Prop_int(dT_m,T_dm[j_h],B,T_p[i],H_e[1][i]))
        
    dT_mag = np.array(dT_high_mag) - np.array(dT_low_mag)
    dT_dmag = np.array(dT_high_dmag) - np.array(dT_low_dmag)
    
    dT_mag_mean = integrate.simps(dT_mag,T_p)/(Th-Tc)
    dT_dmag_mean = integrate.simps(dT_dmag,T_p)/(Th-Tc)
    
    return(dT_mag_mean,dT_dmag_mean)

In [62]:
A1,A2,A3,A4,A5,A6,A7,A8 = Mean_dT(270,300)